In [ ]:
!pip install -q tensorflow tensorflow_hub librosa pandas scipy

In [ ]:
import os
import shutil
import numpy as np
import librosa
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from scipy.signal import find_peaks
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm
import soundfile as sf

In [ ]:
# Cấu hình
SAMPLE_RATE_YAMNET = 16000
SAMPLE_RATE_TRAIN = 32000
WINDOW_SIZE = 0.96
STRIDE = 0.1
SIMILARITY_THRESHOLD = 0.75

In [ ]:
FULL_MATCH_PATH = "/kaggle/input/audio-dataset/full_match.wav" 
TEMPLATE_PATHS = [
    "/kaggle/input/audio-dataset/cut.wav",
    "/kaggle/input/audio-dataset/cut1.wav"
]

# Output folders
OUTPUT_BASE = "/kaggle/working/pickleball_dataset"
MIC_DIR = os.path.join(OUTPUT_BASE, "mic_dev")
META_DIR = os.path.join(OUTPUT_BASE, "metadata_dev")

os.makedirs(MIC_DIR, exist_ok=True)
os.makedirs(META_DIR, exist_ok=True)

print("Đã setup xong môi trường.")

In [ ]:
# Load YAMNet từ TF Hub
print("Đang load YAMNet...")
yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')
print("Đã load YAMNet.")

In [ ]:
def get_embedding(waveform):
    """Lấy vector đặc trưng 1024 chiều từ đoạn âm thanh"""
    scores, embeddings, spectrogram = yamnet_model(waveform)
    
    if len(embeddings) > 0:
        return np.mean(embeddings.numpy(), axis=0).reshape(1, -1)
    return None

In [ ]:
def extract_templates(template_paths):
    """Tạo danh sách embedding từ các file mẫu"""
    templates = []
    for path in template_paths:
        try:
            wav, _ = librosa.load(path, sr=SAMPLE_RATE_YAMNET, mono=True)
            
            if len(wav) > int(WINDOW_SIZE * SAMPLE_RATE_YAMNET):
                center = np.argmax(np.abs(wav))
                start = max(0, center - int(0.48 * SAMPLE_RATE_YAMNET))
                end = start + int(WINDOW_SIZE * SAMPLE_RATE_YAMNET)
                wav = wav[start:end]
            
            if len(wav) < int(WINDOW_SIZE * SAMPLE_RATE_YAMNET):
                wav = np.pad(wav, (0, int(WINDOW_SIZE * SAMPLE_RATE_YAMNET) - len(wav)))
                
            emb = get_embedding(wav)
            if emb is not None:
                templates.append(emb)
                print(f"  + Đã trích xuất đặc trưng mẫu: {os.path.basename(path)}")
        except Exception as e:
            print(f"  x Lỗi file {path}: {e}")
            
    return templates

In [ ]:
def scan_match(full_audio, sr, templates, threshold=0.7):
    """Quét toàn bộ trận đấu để tìm vị trí giống mẫu"""
    detected_times = []
    
    win_len = int(WINDOW_SIZE * sr)
    stride_len = int(STRIDE * sr)
    
    num_steps = (len(full_audio) - win_len) // stride_len
    
    print(f"Bắt đầu quét {len(full_audio)/sr:.1f} giây ({num_steps} bước)...")
    
    scores_over_time = []
    
    # 1. Quét thô (Coarse Scan)
    for i in tqdm(range(num_steps)):
        start_sample = i * stride_len
        end_sample = start_sample + win_len
        chunk = full_audio[start_sample:end_sample]
        
        emb = get_embedding(chunk)
        
        if emb is not None:
            # So sánh với tất cả templates, lấy điểm cao nhất
            max_sim = 0
            for t_emb in templates:
                sim = cosine_similarity(t_emb, emb)[0][0]
                if sim > max_sim:
                    max_sim = sim
            
            scores_over_time.append(max_sim)
            
            # Nếu vượt ngưỡng, lưu lại thời gian thô
            if max_sim >= threshold:
                detected_times.append({
                    'time': i * STRIDE,
                    'score': max_sim,
                    'chunk': chunk
                })
        else:
            scores_over_time.append(0)
            
    return detected_times, np.array(scores_over_time)

In [ ]:
# 1. Load Templates
print("--- Xử lý file mẫu ---")
template_embeddings = extract_templates(TEMPLATE_PATHS)

if not template_embeddings:
    raise ValueError("Không tạo được mẫu nào. Kiểm tra lại file input!")

# 2. Load Full Match
print("\n--- Load trận đấu ---")
y_full, _ = librosa.load(FULL_MATCH_PATH, sr=SAMPLE_RATE_YAMNET, mono=True)

# 3. Quét
print("\n--- Scanning... ---")
raw_hits, score_arr = scan_match(y_full, SAMPLE_RATE_YAMNET, template_embeddings, threshold=SIMILARITY_THRESHOLD)

print(f"\nFound {len(raw_hits)} potential segments.")

# 4. Tinh chỉnh (Refinement) & Lọc trùng (NMS)
print("\n--- Tinh chỉnh vị trí (Peak Picking) ---")

final_labels = []
last_hit_time = -100

raw_hits.sort(key=lambda x: x['score'], reverse=True)

for hit in raw_hits:
    coarse_time = hit['time']
    
    is_duplicate = False
    for existing in final_labels:
        if abs(existing['start'] - coarse_time) < 0.5:
            is_duplicate = True
            break
    if is_duplicate:
        continue
        
    # Tinh chỉnh: Tìm đỉnh năng lượng (Onset) trong đoạn 0.96s
    chunk = hit['chunk']
    onset_env = librosa.onset.onset_strength(y=chunk, sr=SAMPLE_RATE_YAMNET)
    
    local_peaks = librosa.util.peak_pick(onset_env, pre_max=3, post_max=3, pre_avg=3, post_avg=5, delta=0.5, wait=10)
    
    if len(local_peaks) > 0:
        # Lấy peak cao nhất
        best_peak_idx = local_peaks[np.argmax(onset_env[local_peaks])]
        offset_time = librosa.frames_to_time(best_peak_idx, sr=SAMPLE_RATE_YAMNET)
        exact_time = coarse_time + offset_time
    else:
        # Nếu không tìm thấy đỉnh rõ ràng, lấy giữa đoạn
        exact_time = coarse_time + (WINDOW_SIZE / 2)
        
    # Tạo label (độ rộng 0.15s)
    label_entry = {
        'class': 'hit',
        'start': round(exact_time - 0.075, 3),
        'end': round(exact_time + 0.075, 3),
        'ele': 0,
        'azi': 0,
        'score': hit['score']
    }
    final_labels.append(label_entry)

# Sắp xếp lại theo thời gian
final_labels.sort(key=lambda x: x['start'])

print(f"Kết quả cuối cùng: {len(final_labels)} cú đánh được phát hiện.")

In [ ]:
print("\n--- Tạo Dataset Files ---")

base_name = "generated_match_01"

# 1. Lưu CSV
df = pd.DataFrame(final_labels)
if not df.empty:
    csv_path = os.path.join(META_DIR, f"{base_name}.csv")
    df[['class', 'start', 'end', 'ele', 'azi']].to_csv(csv_path, index=False)
    print(f"Đã lưu Metadata: {csv_path}")
    display(df.head())
else:
    print("Không tìm thấy cú đánh nào! Hãy thử giảm SIMILARITY_THRESHOLD.")

# 2. Lưu Audio
wav_out_path = os.path.join(MIC_DIR, f"{base_name}.wav")
print(f"Đang xử lý và lưu Audio (32kHz)...")

y_high, _ = librosa.load(FULL_MATCH_PATH, sr=SAMPLE_RATE_TRAIN, mono=True)
sf.write(wav_out_path, y_high, SAMPLE_RATE_TRAIN)

print(f"Đã lưu Audio: {wav_out_path}")

!zip -r pickleball_dataset.zip {OUTPUT_BASE}
print("\nXONG! Bạn có thể download file pickleball_dataset.zip")